In [1]:
a=list()
for i in range(105):
    a.append(i)

In [2]:
a.append(108)
a.append(107)
a.append(109)
a.append(110)
a.append(113)

In [44]:
import numpy as np
import csv
import pandas

In [38]:
with open('./all_info.csv','rb') as csvfile:
    all_info = pandas.read_csv(csvfile)

In [39]:
ale=all_info.ix[:,a]

In [41]:
t=list()
for i in range(len(ale)):
    if all_info.ix[i,106]>10:
        t.append(1)
    if all_info.ix[i,106]<=10:
        t.append(0)

In [42]:
a=range(110)
a=a[5:]
al=ale.ix[:,a]

In [48]:
import random

In [49]:
train=al

test=np.asarray(t)

train['user_count'][train['user_count'].isnull()]=np.mean(train['user_count'][train['user_count'].notnull()])

train['user_star'][train['user_star'].isnull()]=np.mean(train['user_star'][train['user_star'].notnull()])

train['user_useful'][train['user_useful'].isnull()]=np.mean(train['user_useful'][train['user_useful'].notnull()])

train_t=np.asarray(train)

a=random.sample(range(len(train_t)),150000)

b=list(set(range(len(train_t)))-set(a))

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/pandas/core/generic.py:4059: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3066: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7:

In [116]:
real_train=train_t[a,]
train_labels=test[a,]
valid=train_t[b,]
valid_labels=test[b]

In [52]:
import sklearn.ensemble

In [54]:
clf = sklearn.ensemble.RandomForestClassifier(n_estimators=10)

clf = clf.fit(real_train, train_labels)

In [117]:
ae=clf.predict(valid)

In [62]:
len(t)

229907

In [64]:
ale.shape

(229907, 110)

# my own part - Neural Network

In [225]:
al = np.array(al)

In [226]:
al.shape

(229907, 105)

In [227]:
small_10 = 0
index_small = []
for i in range(len(t)):
    if t[i] == 0:
        small_10 += 1
        index_small.append(i)

In [228]:
small_10

227563

In [229]:
data_nn = al[index_small,:]

In [230]:
data_nn.shape

(227563, 105)

In [269]:
l = np.random.permutation(range(len(data_nn)))
train_i = l[:151708]
valid_i = l[151708:]

In [270]:
train_nn = data_nn[train_i,:]
valid_nn = data_nn[valid_i,:]

In [342]:
from sklearn.preprocessing import MultiLabelBinarizer
all_info = np.array(all_info)
all_y = all_info[:,106]
all_y = np.asarray(all_y, dtype="|S6")
all_y_new = MultiLabelBinarizer().fit_transform(all_y)

In [343]:
train_nn_y = all_y[train_i]
valid_nn_y = all_y[valid_i]

## First Method

In [339]:
from sknn.mlp import Classifier, Layer

In [341]:
nn = Classifier(
    layers=[
        Layer("Tanh", units=50),
        Layer("Sigmoid")],
    learning_rate=0.009,
    n_iter=100)

In [344]:
nn.fit(train_nn[:5000],train_nn_y[:5000])

Classifier(batch_size=1, callback=None, debug=False, dropout_rate=None,
      f_stable=0.001,
      hidden0=<sknn.nn.Layer `Tanh`: name='hidden0', units=50, frozen=False>,
      layers=[<sknn.nn.Layer `Tanh`: name='hidden0', units=50, frozen=False>, <sknn.nn.Layer `Sigmoid`: name='output', units=23, frozen=False>],
      learning_momentum=0.9, learning_rate=0.009, learning_rule='sgd',
      loss_type=None, n_iter=100, n_stable=10, normalize=None,
      output=<sknn.nn.Layer `Sigmoid`: name='output', units=23, frozen=False>,
      parameters=None, random_state=None, regularize=None, valid_set=None,
      valid_size=0.0, verbose=None, warning=None, weight_decay=None)

In [357]:
y_valid = nn.predict(valid_nn[:10000])

[(10000, 23)]


In [416]:
sum(y_valid == b'5')

array([0])

In [345]:
nn.score(valid_nn[:2000],valid_nn_y[:2000])

[(2000, 23)]


0.3755

## Second Method

In [313]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC

In [385]:
nn = OneVsOneClassifier(LinearSVC(random_state=0)).fit(train_nn[:50000],train_nn_y[:50000])

In [387]:
nn.score(valid_nn[:30000],valid_nn_y[:30000])

0.34233333333333332

In [388]:
a = nn.predict(valid_nn[:300])

In [412]:
sum(a == b'2')

40

## Third Method

In [399]:
from sklearn.multiclass import OutputCodeClassifier

In [409]:
cc = OutputCodeClassifier(LinearSVC(random_state=0),code_size=3, random_state=0)

In [410]:
cc.fit(train_nn[:10000],train_nn_y[:10000])

OutputCodeClassifier(code_size=3,
           estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0),
           n_jobs=1, random_state=0)

In [411]:
cc.score(valid_nn[:5000],valid_nn_y[:5000])

0.20080000000000001